DEMOSTRACIÓN

1. Utilizando Vistas

1.1. Accediendo a la base de datos TSQL

In [0]:
USE TSQL;
GO

1.2 Seleccionar y ejecutar el siguiente código para crear una vista

In [0]:
CREATE VIEW HR.EmpPhoneList
AS
SELECT empid, lastname, firstname, phone
FROM HR.Employees;
GO

1.3 Consultar la nueva vista

In [0]:
SELECT empid, lastname, firstname, phone
FROM HR.EmpPhoneList;
GO

1.4 Crear una vista ustilizando un JOIN multi-tabla

In [0]:
CREATE VIEW Sales.OrdersByEmployeeYear
AS
    SELECT  emp.empid AS employee ,
            YEAR(ord.orderdate) AS orderyear ,
            SUM(od.qty * od.unitprice) AS totalsales
    FROM    HR.Employees AS emp
            JOIN Sales.Orders AS ord ON emp.empid = ord.empid
            JOIN Sales.OrderDetails AS od ON ord.orderid = od.orderid
    GROUP BY emp.empid ,
            YEAR(ord.orderdate)
GO

1.5 Seleccionar la vista

In [0]:
SELECT employee, orderyear, totalsales
FROM Sales.OrdersByEmployeeYear
ORDER BY employee, orderyear;

1.6 Limpiar los cambios

In [0]:
DROP VIEW Sales.OrdersByEmployeeYear;
DROP VIEW HR.EmpPhoneList;

2. Utilizando Funciones de Tabla en Linea

2.1. Ejecutar y consultar la siguiente función de ejemplo dbo.GetNums() toma como parámetros: @low (bigint) y @high (bigint)

In [0]:
SELECT * FROM dbo.GetNums(10,20);
GO

2.2. Crear una función para calcular para extensión de líneas para ordenes de compra

In [0]:
CREATE FUNCTION Sales.fn_LineTotal ( @orderid INT )
RETURNS TABLE
AS
RETURN
    SELECT  orderid, productid, unitprice, qty, discount,
            CAST(( qty * unitprice * ( 1 - discount ) ) AS DECIMAL(8, 2)) AS line_total
    FROM    Sales.OrderDetails
    WHERE   orderid = @orderid ;
GO

2.3. Utilizando la función

In [0]:
SELECT orderid, productid, unitprice, qty, discount, line_total
FROM Sales.fn_LineTotal(10252) AS LT;
GO

2.4. Limpiar los cambios

In [0]:
DROP FUNCTION Sales.fn_LineTotal;
GO

3. Utilizando Tablas Derivadas

3.1. Visualizando alias internos de columnas.

In [0]:
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM (SELECT YEAR(orderdate) AS orderyear, custid
FROM Sales.Orders) AS derived_year
GROUP BY orderyear;

3.2. Utilizando una variable como parametro de una tabla derivada

In [0]:
DECLARE @emp_id INT = 9;
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM (	
	SELECT YEAR(orderdate) AS orderyear, custid
	FROM Sales.Orders
	WHERE empid=@emp_id
) AS derived_year
GROUP BY orderyear;

3.3. Enlazando tablas derivadas

In [0]:
SELECT orderyear, cust_count
FROM  (
	SELECT  orderyear, COUNT(DISTINCT custid) AS cust_count
	FROM (
		SELECT YEAR(orderdate) AS orderyear ,custid
        FROM Sales.Orders) AS derived_table_1
	GROUP BY orderyear) AS derived_table_2
WHERE cust_count > 80;

3.4. Una alternativa al ejemplo anterior sería.

In [0]:
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
	FROM (
		SELECT YEAR(orderdate) AS orderyear ,custid
        FROM Sales.Orders) AS derived_table_1
	GROUP BY orderyear
HAVING COUNT(DISTINCT custid) > 80;

4. Utilizando Expresions Comunes de Tabla

4.1. Ejecutar la siguiente expresión.

In [0]:
WITH CTE_year AS
	(
	SELECT YEAR(orderdate) AS orderyear, custid
	FROM Sales.Orders
	)
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM CTE_year
GROUP BY orderyear;

4.2. También se puede utilizar de modo recursico

In [0]:
WITH EmpOrg_CTE AS
(SELECT empid, mgrid, lastname, firstname --anchor query
	FROM HR.Employees
WHERE empid = 5 -- starting "top" of tree. Change this to show other root employees

UNION ALL
SELECT child.empid, child.mgrid, child.lastname, child.firstname -- recursive member which refers back to CTE
	FROM EmpOrg_CTE AS parent
	JOIN HR.Employees AS child
	ON child.mgrid=parent.empid
)
SELECT empid, mgrid, lastname, firstname
FROM EmpOrg_CTE;